# EECS 16B DIY Lab: Expanding on S1XT33N

### EECS 16B: Designing Information Devices and Systems II, Spring 2024

Written by Venkata Alapati (2024)

## Table of Contents

* [Introduction](#intro)
* [Expansion Options](#expansion)
* [Topic 1: PID Control Schemes](#topic1)
* [Topic 2: H-Bridge Motor Circuit](#topic2)


<a id='intro'></a>
## <span style="color:navy">Introduction</span>
-----

Throughout the course of 16B Labs, we've explored a multitude of topics from control schemes to filtering circuits and more. Now, as you reach the end of labs, you have a voice controlled robot car that can do a lot of cool things. But there are always improvements on the horizon and in this "do-it-yourself" extra lab, we will take a look at some of the options you have to improve S1XT33N. This lab and the design contest in general is intended to provide you with an opportunity to choose a topic of your liking and dive deeper than the surface level that was shown in 16B. In the first part, we will talk a little about possible expansion options outside of this notebook. Then we will move into two possible improvements in the control system as well as the motor circuit that allows your S1XT33N to drive backwards!

***Some important notes:***
- **Lab staff will be available during lab sections to discuss ideas with you but they will not be able to provide you with a full design nor will they be able to help you debug your design. ED support will not be available either.**
- **This lab is not mandatory, however extra credit is possible if at least one possible expansion option is explored and a proper writeup is completed. Please refer to ED for more details.**
- **Having fun is mandatory!**

<a id='expansion'></a>
## <span style="color:navy">Expansion Options</span>
-----

If you don't want to follow the topics below, here are some other ideas you can explore sorted by topic:
- **Filters: Color Organ Expansion**
    - **Implement a color organ that can detect different frequencies and light up LEDs based on the sounds detected.**

        Weeeeeeeeeeeee! This semester you have been spared from having to deal with a super noisy lab 5 but while tuning the micboard you have likely heard the annoying tone of a pure sine wave. And in lab 5, we saw how using a combination of Low Pass, High Pass, and Notch Filters with your circuit components could in theory result in frequency detector system which is a solid precursor to the color organ. Now all we need to do is hook up our micboard to the input line and see the sounds light up. This [(non-EECS 16B) video](https://www.youtube.com/watch?v=x5hGF7NsG7Q) does a great job showing what a good final result might be.

        Look through the [datasheet](https://www.ti.com/lit/ds/symlink/lmc6482.pdf) for the op amp. Given this information as well as what you already know about your micboard's behavior, come up with your own filtering / amplifer stags to make your own color organ scheme! 

        Potential ideas include:
        - Adding additional filters to isolate narrower frequency bands
        - Manipulating filter and gain parameters to improve the color organ's response
        - Cascading/combining filters to better isolate frequencies
        - Improving accuracy and brightness in the color organ

        In the end, you can try playing a song in front of the color organ and seeing its LEDs flicker at the different frequencies!
- **AI and Machine Learning**
    - **Use AI and ML techniques to take your S1XT33N to the next level.**

        Fire up your GPU and get those neural networks and CNN crunching data. There is a whole lot of things that can try to do with AI:

        Potential ideas include:
        - Large Language Model Driven Controls -- using LLMS to drive the robot to do certain motion tasks
        - Creating an obstacle detection using a camera based or distance based sensor
        - Implementing true voice classification (likely will need a separate microcontroller board for this)
- **Classification**
    - **Try to build on the current classification model**

        If your lab experience was at all like mine, you know the horrors of screaming into the micboard hundreds of times only to get a classifier that barely works. Its unfortunate but our classifier is quite limited and therefore we challenge you to improve our classifier. A few ideas are listed below:
        - Use a different classification algorithm -- We used clustering but other algorithms may yield better results
        - Expand the number of words you can classify -- Be careful with memory limitations, can you improve this somehow?
        - Reduce the amount of data needed to be stored on the Arduino for classification
- **And there is so much more**


<a id='topic1'></a>

## <span style="color:navy">Topic 1: PID Control Scheme</span>
-----

### Preface

The car's behavior can't not change if I already tried increasing and decreasing the F-values, right???? In Lab 6 and 7, you collected data on your car's driving, fit a linear model to that data, and implemented a closed-loop control scheme to make the car drive in a straight line. However, one limitation of our linear approximation is that it is only accurate for a small range around our center $v_{star}$ value and which is unfortunately susceptible to variation in battery voltage, environmental factors, etc. We challenge you to improve the approximation of the input-velocity curve, such as using a different model (ie quadratic, cubic, etc.) or preprocessing the data to make it more linear. You can also look into other control schemes, such as proportional–integral–derivative (PID) control which we will delve into shortly.

### PID Control Scheme
PID which stands for Proportional, Integral, and Derivative is a common controller and is used in various industrial and commerical applications with the most cited example being cruise control in cars. How this controller works is by taking the difference between our measured output and our desired output and calling that the error term. We then look at this error and feed it back into our controller to get our desired behavior. Let us first conceptually and mathematically understand how this affects our system and how this works to give the desired behavior. 

##### High Level Introduction
The PID controller calculates an error value as the difference between a desired setpoint (SP) and a measured process variable (PV). The controller attempts to minimize the error by adjusting the process control inputs. The PID controller calculation involves three separate parameters; the proportional, the integral and derivative values. The proportional value determines the reaction to the current error, the integral value determines the reaction based on the sum of recent errors, and the derivative value determines the reaction based on the rate at which the error has been changing. For a continuous-time PID controller, the output of the controller is given by the following equation:
<!-- Formula for PID Control -->
**$$u(t) = K_p e(t) + K_i \int_{0}^{t} e(\tau) d\tau + K_d \frac{de(t)}{dt}$$**
<center>

***$e(t)$ is the error term (PV-SP), $K_p$ is the proportional gain, $K_i$ is the integral gain, and $K_d$ is the derivative gain.***

</center>

For a discrete-time PID controller, the output of the controller is given by the following equation:
<!-- Formula for Discrete PID Control -->
**$$u[k] = K_p e[k] + K_i \sum_{j=0}^{k} e[j] + K_d (e[k] - e[k-1])$$**
<center>

***$e(k)$ is the error term (PV-SP), $K_p$ is the proportional gain, $K_i$ is the integral gain, and $K_d$ is the derivative gain.***

</center>

For this lab, we will be working with the discrete-time PID controller and we will slightly tweak our formulation of the PID controller to have our equation focus on ideal distance, velocity, and acceleration (think about why this would work). This yields the following equation:
<!-- Formula for Discrete PID Control -->
**$$u[k] = K_p (v^*[k] - v[k]) + K_i (d^*[k] - d[k]) + K_d (a^*[k] - a[k])$$**
<center>

***$v^*[k]$ is the desired velocity, $v[k]$ is the measured velocity, $d^*[k]$ is the desired distance, $d[k]$ is the measured distance, $a^*[k]$ is the desired acceleration, $a[k]$ is the measured acceleration.***

</center>

##### Proportional
If we just isolate the proportional term we get $K_p*(v^*[i]−v[i])$ and this should be the dominant term of your controller. If we consider what this component is it is saying our input should be higher if we are slower than our target velocity and should be lower if we are faster than our target velocity. the gain term Kp will affect how fast we are going to correct with higher values of Kp leading to us trying to correct faster which will make us reach our target faster, however just like with higher values of F-values from lab 7 this might lead to us over correcting leading to small oscillations before settling or becoming unstable.
##### Integral
If we just isolate the integral term we get $K_i*(d^*[i] -d[i])$ which says we want the wheel to have turned a set amount which we can calculate by $v^*[k]$. This corrects for the steady state error of the velocity, (note that this is different than the steady state delta_ss that we calculated in lab 7) which is the difference in the speed of the wheel after it settles. If we are consistently slower then we would see that the desired distance - distance traveled becomes large and then this integral term would drive our input higher to compensate. So this term drives our system to the desired state accounting for disturbances. Higher values will help us reduce the error from our final ending state, but this can also cause instability and oscillations. Additionally, we should be aware of integrator windup which is when if you get stalled it will cause this term to grow extremely high, larger than reasonable. To account for this we can cap our integral term so that it never grows to large. In the lab we can see this if we hold one of the wheels, what happens when we let go.
##### Derivative
If we just isolate the derivative term we get we just isolate the derivative term we get $K_d*(a^*[i]-a[i])$ which we can think of as a brake. In our car system since we want the acceleration--after the inital startup jolt--to be 0. Hence, we can simplify this to $K_d*(-a[i])$. Similar to the integral term, if our wheel is accelerating faster we will tell it to slow down a little so we don't overshoot, and if our wheel is slowing down we will up our input a little to make sure we don't undershoot. This can be thought of as a damping term that will help correct for oscillatory errors. Since it is damping normally we want a relatively small value of our derivative. Since it is damping normally we want a relatively small value of our derivative gain relative to proportional gain because if this is to high and overpowers our proportional difference then our system won't want to move/ change speeds. Additionally, we should note that the derivatives are often noisy and with discrete data not the best so a lot of sensor error can be introduced in this term.

### Testing Different Implementations 
In addition to the standard PID controller, there are many variations of controllers that can be used to improve performance. Keep in mind that a PWM output for this system is bounded by an 8 bits unsigned integer (0-255). Some of these variations include:
- **Bang Bang Control:** This is the simplest control scheme which is if we are under turn on at full power, and if we are over turn off. This is by nature extremely easy to implement we just need to use an if else statement to send 255 if our measured velocity is below of the desired velocity. This scheme often leads to oscillations that are very visible due to the large swing in inputs.
- **Proportional (P) Control:** This controller only uses the proportional term, taking the difference in our desired velocity (v_star) and current velocity (v_left / v_right), and scales this by some constant of proportionality. We should try to find a good value for Kp that makes our target stable and have the desired behavior before implementing other terms.
- **Proportional-Integral (PI) Control:** By adding the integral component we can reduce our steady state errors so we take our proportional controller and add in the err_Sum_Left / err_Sum_Right terms and multiply this by our Ki value to get our integral term. We then get a controller that goes toward our desired state and corrects for steady state bias.
- **Proportional-Derivative (PD) Control:** For PD Control we can take the difference between our desired acceleration and our measured acceleration and then add this to our proportional control scheme. We multiply the difference in acceleration with our Ki term to be able to get. This controller will allow us to correct for oscillations which allows us to get to our desired state faster without overshooting it.
- **Proportional-Integral-Derivative (PID) Control:** This controller is the most sophisticated, but we can implement this just by adding our found proportional, integral and derivative terms together and setting our output to be the sum of all three terms. This controller tries to both implement the steady state error correction of the PI Control and the oscillation dampening of the PD Control and use both of them.

### Code Implementation
Sample code for the different controllers has been provided with this lab in `pid_sample.ino`. Take a look through the code and try to understand what's happening. Hopefully, you can use this code as a starting point to implement your own PID controller, adding in other cool features like turning! **Make sure to fill in proper values for `CODE BLOCK CON1`.** This will likely involve trial and error to get the desired behavior but there are mathematical ways to help you tune these values as well! 

<a id='topic2'></a>

## <span style="color:navy">Topic 2: H-Bridge Motor Circuit</span>
-----

### Preface 

Wait, can our car drive backwards??? In Lab 3, you built motor controller circuits to take the PWM output from your Arduino and produce an amplified current to power your motors. However, one limitation with the circuit you designed was that current only flows through the motor in one path and hence our motors can only turn "forward". But, what if I told you that you can make your car drive backwards as well?

In this section, we challenge you to fix this limitation via an improved motor controller circuit. A circuit you may find helpful is the [H-bridge](https://en.wikipedia.org/wiki/H-bridge). It may also help your car to make other modifications/improvements as well! There is a lot to choose from including increasing/manipulating speed via resistor sizing modifications, protecting from H-bridge shorts, etc.

**Warning:** ***Be very very very carfeul, we are working with components that could be really dangerous and possibly explode if wired incorrectly. If you have any questions or concerns, please reach out to lab staff! Safety is the #1 priority in any lab!!!!!***

### H-Bridge Motor Circuit
To build our H-Bridge, we will need a few extra components. While a single transistor can be used to control the ON/OFF state of the motor, at least 4 transistors are needed in order to add directionality to the motor. As shown by the image below, the four transistors work in pairs to switch the direction of the current flow through the motor. Here we notice that Q1 and Q4 are on at the same time and Q2 and Q3 are on at the same time. However, you must be very careful when implementing this circuit as if Q1 and Q2 are on at the same time, this will short the power supply/battery and could potentially damage your components.

<!-- Add a centered table showing the 3 different Hbridge images with a caption for each -->
<center>

| Image | Description |
| --- | --- |
| ![H-Bridge](images/h_bridge_path_1.png) | Path 1 through H-Bridge |
| ![H-Bridge](images/h_bridge_path_2.png) | Path 2 through H-Bridge |
| ![H-Bridge](images/h_bridge_short.png) | Short-Circuit Path |

</center>

### Design Considerations
When designing your H-Bridge, you need to consider the components that are available to you. In lab, we have access to <a href="https://www.mouser.com/datasheet/2/308/1/KSC2690A_D-2314385.pdf">C2690A</a> NPN BJT. Traditionally, PNP BJTs are used in the H-Bridge circuit, but we can use NPN BJTs with a few modifications as shown below. Note the difference in resistor values between the top and bottom sets of transistors. This is to ensure that the transistors are actually able to turn on since BJTs need ~0.7V between the base and emitter pins to fully turn on. **The resistor values here are not exact and you may need to adjust them to get the desired behavior.**

<br>
<p align="center">
<img width="600px" src="images/h_bridge_design.png">
</p>
<br>

For reference, the pins for the C2690A and the diodes are as follows:
<span style="color:#ba190f">**Make sure to match the Base, Emitter, and Collector properly to the schematic. Note the diagram shows 3 circles facing up. Additionally, the diode has a proper orientation to it, align the markings correctly**</span>

<br>
<p align="center">
<img width="250px" src="images/proj-bjt.png">
<img width="300px" src="images/proj-diode.png">
</p>
<br>

### Code Implementation
There is no sample code provided for this section as the implementation of the H-Bridge will be mainly hardware based. However, you can reuse the code from Lab 3 to control the H-Bridge. You will need to modify it to add another pin that controls the direction of the motor. I would also strongly encourage you to add some protection logic in your code to prevent the H-Bridge from shorting.

# Congratulations! You have reached the end of the DIY Lab. Don't forget to submit your writeup to Gradescope for extra credit!
### We hope you have enjoyed your time in 16B labs and we hope to see you in future classes!